# Permutation test for ranking data

Can we use permutation tests to test the hypothesis that rankings are different between 2 groups?

Some simulations inspired by the following Stats Exchange question:
https://stats.stackexchange.com/questions/51295/comparison-of-ranked-lists:

> Suppose that two groups, comprising n1n1 and n2n2 each rank a set of 25 items from most to least important. What are the best ways to compare these rankings?

> Clearly, it is possible to do 25 Mann-Whitney U tests, but this would result in 25 test results to interpret, which may be too much (and, in strict use, brings up questions of multiple comparisons). It is also not completely clear to me that the ranks satisfy all the assumptions of this test.

> I would also be interested in pointers to literature on rating vs. ranking.

> Some context: These 25 items all relate to education and the two groups are different types of educators. Both groups are small.

In [1]:
import random

import numpy as np
import toolz as tz

import numpy.linalg

import toyplot as tp

## Functions to generate some data

In [14]:
def isodd(x):
    return x % 2 == 1

def sort_indices(values):
    """
    Return a sorted list of indices based on the values,
    i.e. first element is the index of the smallest value and
    the last element is the index of the largest value
    """
    return sorted(range(len(values)), key=lambda i: values[i])

def generate_groupA(d):
    """
    Group A: small indices have a small advantage
    """
    scores = [random.random() + 0.1*i for i in range(d)]
    return sort_indices(scores)

def generate_groupB(d):
    """
    Group B: small indices and even numbers have small advantage
    """
    scores = [random.random() + 0.1 * i + 0.15 * isodd(i) for i in range(d)]
    return sort_indices(scores)

## Define comparison

In [3]:
def top_vote(s):
    """How often did each option get voted first?"""
    scores = [0 for _ in range(len(s))]
    scores[s[0]] = 1
    return scores

def points(s):
    """
    Aggregate votes by option i getting rank(i) points
    """
    scores = [0 for _ in range(len(s))]
    for i, _ in enumerate(s):
        scores[s[i]] = i
    return scores

def diff_l2(x, y):
    return np.linalg.norm(x-y)

def diff_l1(x, y):
    return np.linalg.norm(x-y, 1)

In [21]:
def compute_diff(groupA, groupB, aggregator, comparator):
    """
    Compute differences between group A and group B
    
    aggregator: how to aggregate the rankings of voters
    comparator: how to compare the outcomes between the aggregations
    of the two groups
    """
    # Compute the mean aggregator function across voters for both groups
    # Returns n_options-dimensional vector for each group of votes
    value_A = np.mean(list(tz.map(aggregator, groupA)), axis=0)
    value_B = np.mean(list(tz.map(aggregator, groupB)), axis=0)

    # Compare the two outputs
    return comparator(value_A, value_B), value_A, value_B

## Permutation test

In [47]:
def permutation_test(groupA, groupB, aggregator, comparator):
    """
    Permutation test: combine, shuffle, split, compute test statistic (repeat)
    """
    aggregated_data = groupA + groupB
    n = len(groupA)
    
    shuffled_data = np.random.permutation(aggregated_data)
    newA = shuffled_data[:n]
    newB = shuffled_data[n:]
    test_statistic, _, _ = compute_diff(newA, newB, aggregator, comparator)
    return test_statistic

def pvalue(Tstar, Tpermutations):
    return sum(1 for T in Tpermutations if T > Tstar) / (len(Tpermutations))

## Generate data and test

In [48]:
number_of_items = 7
number_votersA, number_votersB = 50, 80

votes_A = [generate_groupA(number_of_items) for _ in range(number_votersA)]
votes_B = [generate_groupB(number_of_items) for _ in range(number_votersB)]

In [49]:
aggregator = points
comparator = diff_l1

number_permutations = 1000

# compute test statistic based on generated data
Tstar, valueA, valueB = compute_diff(votes_A, votes_B, aggregator, comparator)
# compute permutation test statistics assuming voters in 2 groups are equal
Tperms = [permutation_test(votes_A, votes_B, aggregator, comparator) for _ in range(number_permutations)]

canvas = tp.Canvas(500, 300)
axes = canvas.cartesian(xlabel="Test statistic", ylabel="Density under null", label="Histogram of permutation test")
axes.bars(np.histogram(Tperms, bins=30, normed=True))
axes.vlines([Tstar], color="red")

0 1 2 3 4 Test statistic 0.0 0.3 0.6 0.9 Density under null Histogram of permutation test 
 
 
 Save as .csv

In [50]:
print("Probability observing T* or more extreme under the null hypothesis: {:.2e}".format(pvalue(Tstar, Tperms)))

Probability observing T* or more extreme under the null hypothesis: 1.00e-03
